<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Isidora Luck
- Nombre de alumno 2:


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/IsidoraLuck/MDS7202.git`

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path="/content/drive/MyDrive/sales.csv"

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv(path)
df['date'] = pd.to_datetime(df['date'])

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [5]:
df.dtypes

id                    int64
date         datetime64[ns]
city                 object
lat                 float64
long                float64
pop                   int64
shop                 object
brand                object
container            object
capacity             object
price               float64
quantity              int64
dtype: object

## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from joblib import dump

In [7]:
# 1. Separar los datos en conjuntos de train, validation y test
semilla = 6340
train, temp = train_test_split(df, test_size=0.3, random_state=semilla)
validation, test = train_test_split(temp, test_size=0.33, random_state=semilla)

# Paso 2: Implementar un FunctionTransformer para extraer día, mes y año de la variable date
def extract_date_info(df):
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df.drop('date', axis=1)

date_transformer = FunctionTransformer(extract_date_info)

# 3. Implementar un ColumnTransformer para procesar datos numéricos y categóricos
numeric_features = ['lat', 'long', 'pop', 'price', 'quantity']
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# 4. Crear un Pipeline con DummyRegressor
pipeline_dummy = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor())
])

# 5. Entrenar el pipeline y reportar la métrica MAE en los datos de validación
pipeline_dummy.fit(train, train['quantity'])
predictions_dummy = pipeline_dummy.predict(validation)
mae_dummy = mean_absolute_error(validation['quantity'], predictions_dummy)
print(f'MAE con DummyRegressor en datos de validación: {mae_dummy}')

# Paso 6: Entrenar el Pipeline con XGBRegressor y reportar el MAE

pipeline_xgb = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

pipeline_xgb.fit(train, train['quantity'])
predictions_xgb = pipeline_xgb.predict(validation)
mae_xgb = mean_absolute_error(validation['quantity'], predictions_xgb)

print(f'MAE con XGBRegressor en datos de validación: {mae_xgb}')

# 7. Guardar los modelos en archivos .pkl
dump(pipeline_dummy, 'pipeline_dummy.pkl')
dump(pipeline_xgb, 'pipeline_xgb.pkl')


MAE con DummyRegressor en datos de validación: 13635.414823264386
MAE con XGBRegressor en datos de validación: 189.04998314810373


['pipeline_xgb.pkl']

La métrica mean_absolute_error (MAE) es una medida comúnmente utilizada para evaluar la precisión de un modelo de regresión, esta mide la diferencia entre la predicción y la realidad de la cantidad demandada, se usa el error absoluto ya que considera el tener más cantidad que la demandada y el tener menos inventario del necesario.


En términos sencillos, un MAE más bajo indica que las predicciones del modelo están más cercanas a los valores reales, lo que generalmente se considera mejor. Por otro lado, un MAE más alto indica que el modelo tiene mayores errores en sus predicciones.

El MAE con XGBRegressor baja mucho en comparación con el MAE del DummyRegressor (13.446 menos de MAE por cantidad de productos demandados menor o mayor a la predecida), teniendo que es mejor el XGBRegressor por disminuir el error en las cantidades demandadas.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [8]:
# 1. Separar los datos en conjuntos de train, validation y test
semilla = 6340
train, temp = train_test_split(df, test_size=0.3, random_state=semilla)
validation, test = train_test_split(temp, test_size=0.33, random_state=semilla)

# 2. Implementar un FunctionTransformer para extraer día, mes y año de la variable date
def extract_date_info(df):
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df.drop('date', axis=1)

date_transformer = FunctionTransformer(extract_date_info)

# 3. Implementar un ColumnTransformer para procesar datos numéricos y categóricos
numeric_features = ['lat', 'long', 'pop', 'price', 'quantity']
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# 6. Entrenar el Pipeline con XGBRegressor y reportar el MAE
# Aplicar restricción monótona en el precio (índice 3 en las características numéricas)
params_constrained = {'monotone_constraints': (0, 0, 0, -1, 0)}

pipeline_xgb_constrain = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(**params_constrained))
])

pipeline_xgb_constrain.fit(train, train['quantity'])
predictions_xgb_constrain = pipeline_xgb_constrain.predict(validation)
mae_xgb_constrain = mean_absolute_error(validation['quantity'], predictions_xgb_constrain)

print(f'MAE con XGBRegressor forzandolo en datos de validación: {mae_xgb_constrain}')

# 7. Guardar los modelos en archivos .pkl
dump(pipeline_xgb_constrain, 'pipeline_xgb_constrain.pkl')


MAE con XGBRegressor forzandolo en datos de validación: 184.02454688934205


['pipeline_xgb_constrain.pkl']

En los resultados obtenidos con esta semilla, el MAE con una correlación negativa (-1) entre el precio y la cantidad hace que el error disminuya (en 5 unidades más o en 5 unidades menos solicitadas), no significativamente respecto al modelo anterior pero si disminuye, teniendo que el amigo tiene razon, lo que es logico pues esto se puede deber a que los productos en general tienen una demanda menor a nivel de cantidades, sin embargo los productos más baratos suelen transarse más (economia basica, demanda aumenta si precio baja), aún así esto siempre puede tener relaciones especiales por factores externos como ciertas modas que afectan la correlación negativa perfecta y no la hacen real.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [9]:
!pip install dill

In [10]:
import optuna
from optuna.samplers import TPESampler
from xgboost import XGBRegressor

# Función objetivo para optimización de hiperparámetros
def objective(trial):
    # Hiperparámetros de XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }

    # Hiperparámetro de OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Actualizar el preprocesador con el nuevo min_frequency
    preprocessor.named_transformers_['cat'].set_params(min_frequency=min_frequency)

    # Crear el Pipeline con XGBRegressor
    pipeline_xgb = Pipeline([
        ('date_transform', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(**xgb_params))
    ])

    # Entrenar y evaluar el modelo
    pipeline_xgb.fit(train, train['quantity'])
    predictions_xgb = pipeline_xgb.predict(validation)
    mae_xgb = mean_absolute_error(validation['quantity'], predictions_xgb)

    return mae_xgb

# Configuración de Optuna
sampler = TPESampler(seed=208)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=1000, timeout=300)  # 5 minutos de tiempo máximo de ejecución

# Mejor configuración encontrada
best_params = study.best_params
best_mae = study.best_value
num_trials = len(study.trials)

print(f'Número de trials: {num_trials}')
print(f'MAE con mejor configuración: {best_mae}')
print('Mejores hiperparámetros:')
print(best_params)

# Extraer el regresor del pipeline existente
xgb_regressor = pipeline_xgb.named_steps['regressor']

# Actualizar el regresor con los mejores parámetros encontrados por Optuna
xgb_regressor.set_params(**best_params)

# Volver a establecer el regresor actualizado en el pipeline
pipeline_xgb.named_steps['regressor'] = xgb_regressor

# Entrenar el pipeline actualizado
pipeline_xgb.fit(train, train['quantity'])

import dill as pickle

# Guardar el modelo con dill
with open('pipeline_optimized.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb, f)


[I 2023-11-17 00:09:49,010] A new study created in memory with name: no-name-4da08577-e12f-43d0-a96e-3772ab973f89
[I 2023-11-17 00:09:50,418] Trial 0 finished with value: 180.7375005867198 and parameters: {'learning_rate': 0.011979739261920794, 'n_estimators': 818, 'max_depth': 3, 'max_leaves': 96, 'min_child_weight': 5, 'reg_alpha': 0.3617398695551255, 'reg_lambda': 0.19594561928608034, 'min_frequency': 0.14061714191117658}. Best is trial 0 with value: 180.7375005867198.
[I 2023-11-17 00:09:58,934] Trial 1 finished with value: 180.5262130105766 and parameters: {'learning_rate': 0.02304458375823557, 'n_estimators': 610, 'max_depth': 9, 'max_leaves': 34, 'min_child_weight': 3, 'reg_alpha': 0.22452215092402006, 'reg_lambda': 0.16333384123691108, 'min_frequency': 0.606287059495959}. Best is trial 1 with value: 180.5262130105766.
[I 2023-11-17 00:10:05,715] Trial 2 finished with value: 186.50917906818466 and parameters: {'learning_rate': 0.09473436171994369, 'n_estimators': 886, 'max_depth

Número de trials: 145
MAE con mejor configuración: 166.90341581744408
Mejores hiperparámetros:
{'learning_rate': 0.009670698800396713, 'n_estimators': 797, 'max_depth': 6, 'max_leaves': 53, 'min_child_weight': 1, 'reg_alpha': 0.2439047530071472, 'reg_lambda': 0.8614830497381458, 'min_frequency': 0.8804639116909209}


In [11]:
with open('pipeline_optimized.pkl', 'rb') as f:
    pipeline_xgb = pickle.load(f)

print(pipeline_xgb)


Pipeline(steps=[('date_transform',
                 FunctionTransformer(func=<function extract_date_info at 0x7c1c461a7c70>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['lat', 'long', 'pop',
                                                   'price', 'quantity']),
                                                 ('cat', OneHotEncoder(),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity',
                                                   'day', 'month', 'year'])])),
                ('regressor',
                 XGBRegressor(base_score=Non...
                              importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.009670698800396713, max_bin=None,
                              max_cat_thresho

**Optimización de hiperparámetros para `XGBRegressor`:**

1. **`learning_rate`:** La tasa de aprendizaje controla la contribución de cada árbol al modelo y ayuda a evitar el sobreajuste. Respecto al rango de optimización, parece razonable el usado, ya que valores muy pequeños pueden requerir más árboles para converger, mientras que valores muy grandes pueden conducir a un sobreajuste.

2. **`n_estimators`:** El número de árboles que se construirán en el modelo. Respecto al rango de optimización, parece razonable el usado, dado que demasiados árboles pueden conducir a un sobreajuste, mientras que muy pocos pueden resultar en un subajuste.

3. **`max_depth`:** La profundidad máxima de cada árbol. El rango limita la complejidad de cada árbol. Valores demasiado altos pueden llevar a sobreajuste. En este aspecto parece bien el árbol.

4. **`max_leaves`:** El número máximo de nodos terminales (hojas) en cada árbol. El rango es una opción razonable. Limitar el número de hojas puede ayudar a controlar la complejidad del modelo.

5. **`min_child_weight`:** La suma mínima de pesos (hessiano) necesarios en un hijo (nodos). EL rango de optimización controla la creación de nodos hijos. Valores más altos conducen a una mayor conservación y menos ramificación.

6. **`reg_alpha` y `reg_lambda`:** Son los términos de regularización que controlan la penalización por la complejidad del modelo. Valores más altos penalizan modelos más complejos, por lo que están bien los usados.

**Optimización de hiperparámetros para `OneHotEncoder`:**

1. **`min_frequency`:** La frecuencia mínima requerida para una categoría para ser considerada en la codificación one-hot. Controla la inclusión de categorías infrecuentes. Un valor más alto puede reducir la dimensionalidad si hay muchas categorías poco frecuentes.


Número de trials: 145

MAE con mejor configuración: 166.90341581744408

Mejores hiperparámetros:
{'learning_rate': 0.009670698800396713, 'n_estimators': 797, 'max_depth': 6, 'max_leaves': 53, 'min_child_weight': 1, 'reg_alpha': 0.2439047530071472, 'reg_lambda': 0.8614830497381458, 'min_frequency': 0.8804639116909209}

El cambio en el MAE de una baja en 17 a 18 respecto al anterior, se debe al uso de metodos de optimización de hiperparametros con optuna, esto permite que el modelo itere encontrando la mejor estrategía que puede adoptar luego de iterar 145 veces en 5 minutos, estos resultados siempre se debe comparar respecto al conjunto de test, ya que puede sobreajustarse a los datos.

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

La pruning corresponde a la poda, es decir, es un proceso de eliminación o restricción de algunas partes del árbol (ramas, nodos hijos) con el objetivo de mejorar la generalización del modelo y evitar el sobreajuste.

Su impacto se ve reflejado principalmente en la reducción del sobreajuste, puesto que la poda ayuda a limitar la complejidad del árbol. Árboles no podados tienden a ajustarse demasiado a los detalles específicos de los datos de entrenamiento, lo que puede resultar en un rendimiento deficiente en datos no vistos. De esta misma manera se mejora la generalización por la limitación de la profundidad o el número de nodos de los árboles.


In [12]:
import optuna
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from joblib import dump
import pandas as pd

# Asegúrate de que df esté definido y sea tu DataFrame original

# Convertir columnas a categorías y extraer información de la fecha
def extract_and_convert_to_category(df):
    # Comprobar si 'date' está en las columnas del DataFrame
    if 'date' in df.columns:
        # Si 'date' existe, extraer 'day', 'month' y 'year' y convertirlos a categorías
        df['day'] = df['date'].dt.day.astype('category')
        df['month'] = df['date'].dt.month.astype('category')
        df['year'] = df['date'].dt.year.astype('category')
        df = df.drop('date', axis=1)  # Eliminar la columna 'date' después de extraer la información
# Convertir otras columnas a categorías
    for col in ['city', 'shop', 'brand', 'container', 'capacity']:
        if col in df.columns:  # Comprobar si cada columna existe antes de convertirla
            df[col] = df[col].astype('category')

    return df


date_transformer = FunctionTransformer(extract_and_convert_to_category)

# Aplicar la transformación a los datos
train = extract_and_convert_to_category(train)
validation = extract_and_convert_to_category(validation)

# Definición de la función objetivo para Optuna
def objective(trial):
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }

    pipeline_xgb = Pipeline([
        ('date_transform', date_transformer),
        ('regressor', XGBRegressor(**xgb_params, enable_categorical=True))
    ])

    pruning_callback = XGBoostPruningCallback(trial, 'validation_0-mae')

    pipeline_xgb.fit(train, train['quantity'],
                     regressor__eval_set=[(validation, validation['quantity'])],
                     regressor__early_stopping_rounds=50,
                     regressor__eval_metric='mae',
                     regressor__callbacks=[pruning_callback])


    predictions_xgb = pipeline_xgb.predict(validation)
    mae_xgb = mean_absolute_error(validation['quantity'], predictions_xgb)

    return mae_xgb

# Proceso de optimización con Optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
sampler = TPESampler(seed=208)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=1000, timeout=300, show_progress_bar=True)

best_params = study.best_params
best_mae_op = study.best_value
num_trials = len(study.trials)

print(f'Número de trials: {num_trials}')
print(f'MAE con mejor configuración: {best_mae_op}')
print('Mejores hiperparámetros:')
print(best_params)

# Asumiendo que pipeline_xgb tiene un paso llamado 'regressor' que es un XGBRegressor
# y best_params es un diccionario con los mejores parámetros encontrados por Optuna

# Modificar las claves en best_params para que coincidan con el formato del pipeline
best_params_reformatted = {'regressor__' + key: value for key, value in best_params.items()}


# Establecer los parámetros reformateados en el pipeline
# Crear un nuevo pipeline con los parámetros optimizados
pipeline_optimized = Pipeline([
    ('date_transform', date_transformer),
    ('regressor', XGBRegressor(**best_params_reformatted, enable_categorical=True))
])

# Entrenar el pipeline optimizado
pipeline_optimized.fit(train, train['quantity'])

# Guardar el modelo optimizado
dump(pipeline_optimized, 'pipeline_optimized.pkl')


  0%|          | 0/1000 [00:00<?, ?it/s]

[0]	validation_0-mae:13473.94096
[1]	validation_0-mae:13313.27726
[2]	validation_0-mae:13156.17521
[3]	validation_0-mae:12999.89964
[4]	validation_0-mae:12846.26742
[5]	validation_0-mae:12693.26237
[6]	validation_0-mae:12541.58268
[7]	validation_0-mae:12392.61820
[8]	validation_0-mae:12245.81736
[9]	validation_0-mae:12101.86027
[10]	validation_0-mae:11958.06456
[11]	validation_0-mae:11818.57299
[12]	validation_0-mae:11678.26498
[13]	validation_0-mae:11542.11508
[14]	validation_0-mae:11406.07831
[15]	validation_0-mae:11271.26277
[16]	validation_0-mae:11138.52017
[17]	validation_0-mae:11009.35085
[18]	validation_0-mae:10878.37660


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[19]	validation_0-mae:10752.17629
[20]	validation_0-mae:10626.01328
[21]	validation_0-mae:10502.64670
[22]	validation_0-mae:10378.66687
[23]	validation_0-mae:10259.06104
[24]	validation_0-mae:10137.64938
[25]	validation_0-mae:10020.93318
[26]	validation_0-mae:9903.42361
[27]	validation_0-mae:9787.75619
[28]	validation_0-mae:9675.37311
[29]	validation_0-mae:9560.85184
[30]	validation_0-mae:9450.81794
[31]	validation_0-mae:9341.23155
[32]	validation_0-mae:9234.11946
[33]	validation_0-mae:9126.25060
[34]	validation_0-mae:9020.69537
[35]	validation_0-mae:8913.78446
[36]	validation_0-mae:8810.45767
[37]	validation_0-mae:8709.00241
[38]	validation_0-mae:8607.73409
[39]	validation_0-mae:8508.36163
[40]	validation_0-mae:8410.19328
[41]	validation_0-mae:8311.99261
[42]	validation_0-mae:8213.92164
[43]	validation_0-mae:8119.48155
[44]	validation_0-mae:8024.63534
[45]	validation_0-mae:7932.41224
[46]	validation_0-mae:7837.64121
[47]	validation_0-mae:7747.16896
[48]	validation_0-mae:7657.48075
[49

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


...

[8]	validation_0-mae:6192.73949
[9]	validation_0-mae:5674.98762
[10]	validation_0-mae:5199.61451
[11]	validation_0-mae:4766.47034
[12]	validation_0-mae:4367.92358
[13]	validation_0-mae:4002.17379
[14]	validation_0-mae:3666.29336
[15]	validation_0-mae:3360.81928
[16]	validation_0-mae:3082.47844
[17]	validation_0-mae:2826.82813
[18]	validation_0-mae:2592.38208
[19]	validation_0-mae:2377.86407
[20]	validation_0-mae:2181.31091
[21]	validation_0-mae:2001.61094
[22]	validation_0-mae:1836.21103
[23]	validation_0-mae:1685.99475
[24]	validation_0-mae:1547.90613
[25]	validation_0-mae:1420.81455
[26]	validation_0-mae:1306.46223
[27]	validation_0-mae:1200.97588
[28]	validation_0-mae:1103.91644
[29]	validation_0-mae:1016.99049
[30]	validation_0-mae:936.51453
[31]	validation_0-mae:862.09605
[32]	validation_0-mae:795.26414
[33]	validation_0-mae:734.02083
[34]	validation_0-mae:678.21300
[35]	validation_0-mae:628.71746
[36]	validation_0-mae:582.85018
[37]	validation_0-mae:541.88375
[38]	validation_0-ma

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[6]	validation_0-mae:7141.44485
[7]	validation_0-mae:6512.18062
[8]	validation_0-mae:5940.54896
[9]	validation_0-mae:5416.78476
[10]	validation_0-mae:4942.22295
[11]	validation_0-mae:4509.57407
[12]	validation_0-mae:4113.27919
[13]	validation_0-mae:3751.08502
[14]	validation_0-mae:3422.55940
[15]	validation_0-mae:3123.82263
[16]	validation_0-mae:2851.95894
[17]	validation_0-mae:2605.05151
[18]	validation_0-mae:2378.26433
[19]	validation_0-mae:2172.22173
[20]	validation_0-mae:1983.17015
[21]	validation_0-mae:1811.93735
[22]	validation_0-mae:1655.75418
[23]	validation_0-mae:1513.29047
[24]	validation_0-mae:1384.43246
[25]	validation_0-mae:1266.80148
[26]	validation_0-mae:1159.49812
[27]	validation_0-mae:1061.84899
[28]	validation_0-mae:972.47788
[29]	validation_0-mae:891.62238
[30]	validation_0-mae:818.51740
[31]	validation_0-mae:752.56134
[32]	validation_0-mae:693.40975
[33]	validation_0-mae:639.10580
[34]	validation_0-mae:590.15143
[35]	validation_0-mae:545.70733
[36]	validation_0-mae:

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [00:19:53] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "regressor__learning_rate", "regressor__max_depth", "regressor__max_leaves", "regressor__min_child_weight", "regressor__n_estimators", "regressor__reg_alpha", "regressor__reg_lambda" } are not used.

  warnings.warn(smsg, UserWarning)


['pipeline_optimized.pkl']

Número de trials: 123

MAE con mejor configuración: 162.09654278621494

Mejores hiperparámetros:
{'learning_rate': 0.07680093460552403, 'n_estimators': 190, 'max_depth': 10, 'max_leaves': 88, 'min_child_weight': 1, 'reg_alpha': 0.6758615414620017, 'reg_lambda': 0.7834775832102502}

La mejora del MAE de una baja en 4 respecto al anterior se debe al uso de la poda, el modelo encuentra su mejor version que puede adoptar en 5 minutos luego de iterar 123 veces.

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [13]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [20]:
plot_optimization_history(study)

In [21]:
plot_parallel_coordinate(study)

In [22]:
plot_param_importances(study)

Al observar el primer grafico, se tiene que hay alrededor de 4 bajadas significatibas, teniendo que la mejora respecto al caso anterior se evidencia en el trial 49, luego de esto el costo computacional es muy alto para la variación del error.

En el segundo grafico se puede observar que las principales tendencias apuntan a que el learning rate es más alto para los mejores resultados, en general para los intentos de máx depthse usan valores altos como 9 o 10, de manera similar ocurre para max_leaves donde se usan valores de 80 a 100, teniendo que el min_child_weight suele ser 1, el n_estimators es el más variable, sin embargo sus mejores resultados son los casos que usan un menor número cercalo a 50 y no a 1000, reg_alpha ronda los 0.6, de manera similar reg_lambda ronda eso valores hasta 0.8.

Finalmente, en el tercer grafico se observa que los hiperparametros más relevantes son el learning rate y el n_estimators, es decir la tasa de aprendizaje del modelo y la cantidad de intentos que tiene para modelar aleatoriamente el árbol.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento?

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [24]:
from IPython.display import display

# Crear un diccionario con la información de los modelos y sus MAE
results = {
    'Modelo': ['DummyRegressor', 'XGBoost', 'XGBoost con Constraints', 'XGBoost con Optuna', 'XGBoost Optimizado y Pruning'],
    'MAE en Validación': [mae_dummy, mae_xgb, mae_xgb_constrain, best_mae, best_mae_op],
}

# Crear un DataFrame a partir del diccionario
summary_table = pd.DataFrame(results)

# Mostrar la tabla resumen
display(summary_table)

,Modelo,MAE en Validación
0,DummyRegressor,13635.414823
1,XGBoost,189.049983
2,XGBoost con Constraints,184.024547
3,XGBoost con Optuna,166.903416
4,XGBoost Optimizado y Pruning,162.096543


El mejor modelo corresponde al último, en el que se hace optimización de hiperparametros y se hace poda a los árboles generados.

In [18]:
from joblib import load

# Cargar el mejor modelo
best_model = load('pipeline_optimized.pkl')

# Predecir sobre el conjunto de prueba
predictions_test = best_model.predict(test)

# Calcular el MAE en el conjunto de prueba
mae_test = mean_absolute_error(test['quantity'], predictions_test)

# Imprimir el MAE en el conjunto de prueba
print(f'MAE en datos de prueba: {mae_test}')


MAE en datos de prueba: 162.37289920130667


El MAE respecto al conjunto de validación aumenta, sin embargo aumenta a nivel de decimales (casi nada), esto se debe a que el modelo no se subajusto ni se sobreajusto a los datos teniendo poca diferencia del error entre validación y test, esto ocurre cuando se optimizan los hiperparametros como se hizo en este caso, aún así las semillas usadas debieron ser buenas ya que ayudaron a disminuir el error y las diferencias del error de validación y test.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>